In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
import torch
from torch import nn

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
train.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [6]:
# let's separate numeric columns and categorical columns 
num_cols = train.select_dtypes(include=np.number).columns.tolist()
num_cols.remove('id')
cat_cols = train.select_dtypes(include = ['object', 'string']).columns.tolist()

In [7]:
num_features = ['study_hours', 'class_attendance', 'sleep_hours']
cat_features = cat_cols   # your categorical columns

X = train[num_features + cat_features]
y = train['exam_score']

In [8]:
X = pd.get_dummies(X, drop_first=True)
X = X.astype(int)
X.head()

,study_hours,class_attendance,sleep_hours,gender_male,gender_other,course_b.sc,course_b.tech,course_ba,course_bba,course_bca,...,sleep_quality_good,sleep_quality_poor,study_method_group study,study_method_mixed,study_method_online videos,study_method_self-study,facility_rating_low,facility_rating_medium,exam_difficulty_hard,exam_difficulty_moderate
0,7,98,4,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
1,4,94,4,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
2,4,92,5,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,2,49,8,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,7,86,9,1,0,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [10]:
# Scale the inputs
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# convert to tensor 

# X Training data
X_train =  torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

# y training data 
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1,1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1,1)

In [12]:
# device agnostic code
device ="cuda" if torch.cuda.is_available() else "cpu"

In [13]:
X.shape

(630000, 22)

In [20]:
# create a neural network 
class ExamScore(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)



model_0 = ExamScore(in_features=X.shape[1])
model_0.to(device)

ExamScore(
  (net): Sequential(
    (0): Linear(in_features=22, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [26]:
# define loss function and optimizer 
loss_fn = nn.L1Loss()

# set optimizer
optimizer = torch.optim.SGD(model_0.parameters(), lr=0.02)

In [24]:
epochs = 2000
patience = 20   # stop if no improvement for 20 epochs

best_test_loss = float("inf")
epochs_no_improve = 0

for epoch in range(epochs):

    # ----- TRAIN -----
    model_0.train()
    y_pred = model_0(X_train)
    train_loss = loss_fn(y_pred, y_train)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # ----- TEST -----
    model_0.eval()
    with torch.inference_mode():
        test_pred = model_0(X_test)
        test_loss = loss_fn(test_pred, y_test)

    # ----- EARLY STOPPING CHECK (EVERY EPOCH) -----
    if test_loss.item() < best_test_loss:
        best_test_loss = test_loss.item()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

    # ----- LOGGING -----
    if epoch % 100 == 0:
        print(
            f"Epoch {epoch} | "
            f"Train Loss: {train_loss.item():.4f} | "
            f"Test Loss: {test_loss.item():.4f}"
        )
        print("-" * 50)


Epoch 0 | Train Loss: 7.2366 | Test Loss: 7.2625
--------------------------------------------------
Epoch 100 | Train Loss: 7.2313 | Test Loss: 7.2573
--------------------------------------------------
Epoch 200 | Train Loss: 7.2272 | Test Loss: 7.2527
--------------------------------------------------
Epoch 300 | Train Loss: 7.2237 | Test Loss: 7.2490
--------------------------------------------------
Epoch 400 | Train Loss: 7.2206 | Test Loss: 7.2462
--------------------------------------------------
Epoch 500 | Train Loss: 7.2180 | Test Loss: 7.2440
--------------------------------------------------
Epoch 600 | Train Loss: 7.2162 | Test Loss: 7.2426
--------------------------------------------------
Epoch 700 | Train Loss: 7.2146 | Test Loss: 7.2416
--------------------------------------------------
Epoch 800 | Train Loss: 7.2132 | Test Loss: 7.2405
--------------------------------------------------
Epoch 900 | Train Loss: 7.2119 | Test Loss: 7.2395
---------------------------------

In [27]:
model_1 = ExamScore(in_features=X.shape[1])
model_1.to(device)

ExamScore(
  (net): Sequential(
    (0): Linear(in_features=22, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [28]:
epochs = 2000
patience = 20   

best_test_loss = float("inf")
epochs_no_improve = 0

for epoch in range(epochs):

    # ----- TRAIN -----
    model_0.train()
    y_pred = model_0(X_train)
    train_loss = loss_fn(y_pred, y_train)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # ----- TEST -----
    model_0.eval()
    with torch.inference_mode():
        test_pred = model_0(X_test)
        test_loss = loss_fn(test_pred, y_test)

    # ----- EARLY STOPPING CHECK (EVERY EPOCH) -----
    if test_loss.item() < best_test_loss:
        best_test_loss = test_loss.item()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

    # ----- LOGGING -----
    if epoch % 100 == 0:
        print(
            f"Epoch {epoch} | "
            f"Train Loss: {train_loss.item():.4f} | "
            f"Test Loss: {test_loss.item():.4f}"
        )
        print("-" * 50)


Epoch 0 | Train Loss: 7.1973 | Test Loss: 7.2246
--------------------------------------------------
Epoch 100 | Train Loss: 7.1973 | Test Loss: 7.2246
--------------------------------------------------
Epoch 200 | Train Loss: 7.1973 | Test Loss: 7.2246
--------------------------------------------------
Epoch 300 | Train Loss: 7.1973 | Test Loss: 7.2245
--------------------------------------------------
Epoch 400 | Train Loss: 7.1973 | Test Loss: 7.2245
--------------------------------------------------
Epoch 500 | Train Loss: 7.1973 | Test Loss: 7.2245
--------------------------------------------------
Epoch 600 | Train Loss: 7.1973 | Test Loss: 7.2245
--------------------------------------------------
Epoch 700 | Train Loss: 7.1973 | Test Loss: 7.2245
--------------------------------------------------
Epoch 800 | Train Loss: 7.1973 | Test Loss: 7.2245
--------------------------------------------------
Epoch 900 | Train Loss: 7.1973 | Test Loss: 7.2245
---------------------------------

In [23]:
test = test[num_features + cat_features]
X = pd.get_dummies(test, drop_first=True)
X = X.astype(int)
X.head()

,study_hours,class_attendance,sleep_hours,gender_male,gender_other,course_b.sc,course_b.tech,course_ba,course_bba,course_bca,...,sleep_quality_good,sleep_quality_poor,study_method_group study,study_method_mixed,study_method_online videos,study_method_self-study,facility_rating_low,facility_rating_medium,exam_difficulty_hard,exam_difficulty_moderate
0,6,65,5,0,1,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
1,6,45,9,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,6,98,6,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,1,0,1
3,3,66,5,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
4,2,42,9,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [28]:
X_test_kaggle = scaler.transform(X)

X_test_kaggle = torch.from_numpy(X_test_kaggle).float()
X_test_kaggle = X_test_kaggle.to(device)

model_0.eval()

with torch.no_grad():
    predictions = model_0(X_test_kaggle)

predictions = predictions.cpu().numpy().flatten()

In [29]:
submission = pd.read_csv("sample_submission.csv")
submission["exam_score"] = predictions   # use target column name
submission.to_csv("submission.csv", index=False)